# Data from 1.11.2021 to the end of the app

In [89]:
import pandas as pd
import numpy as np
import bz2
pd.set_option('display.max_columns', None)
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import matplotlib as mpl
from scipy import stats
from distfit import distfit
import scikit_posthocs as sp
import datetime as dt

pd.options.display.float_format = '{:.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [90]:
df = pd.read_csv("/home/evida-monika/mhunters/final_data_3.csv.bz2", compression="bz2", sep=",")

# this is cleaned dataset
# we need to delete another user - ID = 235, because it has 274 sessions in 4 weeks - impossible.

df = df[df['id_users'] != 235]

In [91]:
# changing types of data

cols = ['name_en_exercises','discarded_session_execution', 'code_name_sessions', 
        'name_en_sessions', 'active_user_programs', 'completed_user_programs',
        'pro_programs', 'available_programs', 'name_en_programs', 
        'gender', 
        'activity_level', 'goal', 'body_type', 'newsletter_subscription', 'notifications_setting',
        'language', 'scientific_data_usage', 'BMI_category', 'name_en_implements']

for col in cols:
    df[col] = df[col].astype('category')
    
col_date = ['updated_at_ex_ex', 'created_at_exercises', 'updated_at_exercises',
            'updated_at_session_execution', 'created_at_user_programs', 'updated_at_user_programs',
            'created_at_programs', 'updated_at_programs', 'created_at_users', 'updated_at_users',
            'date_of_birth']

for col in col_date:
    df[col] = pd.to_datetime(df[col])#.dt.strftime("%Y-%m-%d %H:%M:%S")

cat_names = {
    'gender': {1: 'male', 0: 'female'},
    'activity_level': {0: 'very active', 1: 'active', 2: 'sedentary'},
    'goal': {0: 'lose', 1: 'gain', 2: 'antiaging'},
    'body_type': {0: 'thin', 1: 'mid', 2: 'strong'}
}

df = df.replace(cat_names)


In [92]:
# dataset only for users

df_users_only_once = df.drop_duplicates(subset=['id_users'], keep='first')

In [93]:
# here dropping all of the columns that are not relevant for users

df_users_only = df_users_only_once.loc[:, ['id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
                                          'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
                                          'total_calories_users2', 'kcal_per_session_users2']]

In [94]:
# dataset for only one of id_session_execution

df_session_exe_only_once = df.drop_duplicates(subset=['id_session_execution'], keep='first')

In [95]:
# here dropping all of the columns that are not relevant for session_execution

df_session_exe = df_session_exe_only_once.loc[:, ['id_session_execution', 
                                                  'difficulty_feedback_session_execution',
       'enjoyment_feedback_session_execution',
       'reps_executed_session_execution', 'updated_at_session_execution',
       'discarded_session_execution', 'id_sessions', 'order_sessions',
       'time_duration_sessions', 'code_name_sessions', 'name_en_sessions',
       'id_user_programs', 'created_at_user_programs',
       'updated_at_user_programs', 'active_user_programs',
       'current_session_id_user_programs', 'completed_user_programs',
       'id_programs', 'created_at_programs', 'updated_at_programs',
       'pro_programs', 'available_programs', 'strength_programs',
       'endurance_programs', 'technique_programs', 'flexibility_programs',
       'intensity_programs', 'name_en_programs', 'description_en_programs',
       'id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
       'total_time_session_execution', 'total_reps_session_execution',
       'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
       'total_time_session_execution_min', 'reps_per_min_session_execution',
       'exercise_execution_time_min', 'calories_session_execution',
       'total_calories_users2', 'kcal_per_session_users2',
                                          'YYYY/WW']]

In [96]:
# manipulations

users_import = pd.read_csv("/home/evida-monika/mhunters/users.csv", sep=",")

users_import = users_import.loc[:, ['id', 'imported']]

users_import.replace('t', True, inplace = True)
users_import.replace('f', False, inplace = True)

# users_import_true = users_import[users_import['imported'] == True]

# imported_true = df_users_only_once.merge(users_import_true, how = 'inner', left_on = 'id_users', right_on = 'id')

users_import2 = users_import[users_import['imported'] == False]

# users_import.shape

imported = df_users_only_once.merge(users_import, how = 'inner', left_on = 'id_users', right_on = 'id')

# df_users_only.shape

imported.shape

(3171, 84)

In [97]:
imported_all = df.merge(users_import, how = 'inner', left_on = 'id_users', right_on = 'id')

df.shape

(1860393, 82)

In [98]:
imported_all

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,YYYY/WW,id,imported
0,342539,0.00,0.00,14,2020-12-11 14:17:16.127382,11758.00,2.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,1291.00,1.00,1291.00,5.00,3.00,75.00,2020-12-11 14:17:15.877294,False,88.00,1.00,996.00,PH2-2-1,IRP & IR,659.00,2020-12-11 12:57:19.338749,2020-12-11 12:57:19.338749,True,88.00,False,17.00,2020-11-23 13:57:13.225121,2021-09-29 14:53:44.706540,False,True,2.00,2.00,1.00,2.00,2.00,Smash your goals,"Get into the habit of working out, with easy s...",182.00,2020-12-11 11:34:00.850343,2021-12-22 06:39:39.280245,male,2000-01-01,160.00,60.00,active,lose,thin,15.00,False,True,1.00,es,220.00,False,1.00,0.00,No implement,23.44,Normal,2.00,338.00,81.00,583.00,131.00,65.50,5.63,14.38,0.00,0.00,16.02,26.30,13.15,2020/50,182,False
1,342538,0.00,0.00,14,2020-12-11 14:17:16.122598,11758.00,2.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,1291.00,1.00,1291.00,5.00,3.00,75.00,2020-12-11 14:17:15.877294,False,88.00,1.00,996.00,PH2-2-1,IRP & IR,659.00,2020-12-11 12:57:19.338749,2020-12-11 12:57:19.338749,True,88.00,False,17.00,2020-11-23 13:57:13.225121,2021-09-29 14:53:44.706540,False,True,2.00,2.00,1.00,2.00,2.00,Smash your goals,"Get into the habit of working out, with easy s...",182.00,2020-12-11 11:34:00.850343,2021-12-22 06:39:39.280245,male,2000-01-01,160.00,60.00,active,lose,thin,15.00,False,True,1.00,es,220.00,False,1.00,0.00,No implement,23.44,Normal,2.00,338.00,81.00,583.00,131.00,65.50,5.63,14.38,0.00,0.00,16.02,26.30,13.15,2020/50,182,False
2,342537,0.00,0.00,14,2020-12-11 14:17:16.118512,11758.00,2.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,1291.00,1.00,1291.00,5.00,3.00,75.00,2020-12-11 14:17:15.877294,False,88.00,1.00,996.00,PH2-2-1,IRP & IR,659.00,2020-12-11 12:57:19.338749,2020-12-11 12:57:19.338749,True,88.00,False,17.00,2020-11-23 13:57:13.225121,2021-09-29 14:53:44.706540,False,True,2.00,2.00,1.00,2.00,2.00,Smash your goals,"Get into the habit of working out, with easy s...",182.00,2020-12-11 11:34:00.850343,2021-12-22 06:39:39.280245,male,2000-01-01,160.00,60.00,active,lose,thin,15.00,False,True,1.00,es,220.00,False,1.00,0.00,No implement,23.44,Normal,2.00,338.00,81.00,583.00,131.00,65.50,5.63,14.38,0.00,0.00,16.02,26.30,13.15,2020/50,182,False
3,342536,0.00,0.00,14,2020-12-11 14:17:16.114356,11758.00,2.00,5968,2020-10-16 09:33:15.172657,2022-01-21 

In [99]:
imported_all.shape

imported_sessions = df_session_exe.merge(users_import, how = 'inner', left_on = 'id_users', right_on = 'id')

In [100]:
imported_sessions.head()

,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,BMI,BMI_category,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories_session_execution,total_calories_users2,kcal_per_session_users2,YYYY/WW,id,imported
0,1291.00,5.00,3.00,75.00,2020-12-11 14:17:15.877294,False,88.00,1.00,996.00,PH2-2-1,IRP & IR,659.00,2020-12-11 12:57:19.338749,2020-12-11 12:57:19.338749,True,88.00,False,17.00,2020-11-23 13:57:13.225121,2021-09-29 14:53:44.706540,False,True,2.00,2.00,1.00,2.00,2.00,Smash your goals,"Get into the habit of working out, with easy s...",182.00,2020-12-11 11:34:00.850343,2021-12-22 06:39:39.280245,male,2000-01-01,160.00,60.00,active,lose,thin,15.00,False,True,1.00,es,220.00,False,1.00,23.44,Normal,2.00,338.00,81.00,583.00,131.00,65.50,5.63,14.38,0.00,16.02,26.30,13.15,2020/50,182,False
1,1292.00,5.00,3.00,45.00,2020-12-11 14:22:50.864652,False,88.00,1.00,996.00,PH2-2-1,IRP & IR,659.00,2020-12-11 12:57:19.338749,2020-12-11 12:57:19.338749,True,88.00,False,17.00,2020-11-23 13:57:13.225121,2021-09-29 14:53:44.706540,False,True,2.00,2.00,1.00,2.00,2.00,Smash your goals,"Get into the habit of working out, with easy s...",182.00,2020-12-11 11:34:00.850343,2021-12-22 06:39:39.280245,male,2000-01-01,160.00,60.00,active,lose,thin,15.00,False,True,1.00,es,220.00,False,1.00,23.44,Normal,2.00,245.00,50.00,583.00,131.00,65.50,4.08,12.24,0.02,10.28,26.30,13.15,2020/50,182,False
2,3253.00,5.00,3.00,0.00,2021-02-09 12:53:19.873117,False,223.00,1.00,1405.00,PM5-2-1,IRP & TB,704.00,2021-02-02 11:11:57.575351,2021-03-17 12:35:12.690296,False,226.00,False,27.00,2020-11-23 14:05:30.101469,2021-09-29 14:53:35.625344,False,True,3.00,4.00,2.00,3.00,4.00,Smash your goals,The 10-session program created to help you bui...,360.00,2021-02-02 11:10:55.159616,2021-12-22 06:39:41.148120,female,2000-01-01,160.00,60.00,sedentary,lose,thin,25.00,False,True,5.00,en,1300.00,False,10.00,23.44,Normal,10.00,97.00,30.00,7697.00,2106.00,210.60,1.62,18.56,0.02,3.64,439.54,43.95,2021/06,360,False
3,3279.00,5.00,3.00,282.00,2021-03-14 11:44:54.066742,False,224.00,2.00,988.00,PM5-2-2,SS & IT,704.00,2021-02-02 11:11:57.575351,2021-03-17 12:35:12.690296,False,226.00,False,27.00,2020-11-23 14:05:30.101469,2021-09-29 14:53:35.625344,False,True,3.00,4.00,2.00,3.00,4.00,Smash your goals,The 10-session program created to help you bui...,360.00,2021-02-02 11:10:55.159616,2021-12-22 06:39:41.148120,female,2000-01-01,160.00,60.00,sedentary,lose,thin,25.00,False,True,5.00,en,1300.00,False,10.00,23.44,Normal,10.00,1333.00,282.00,7697.00,2106.00,210.60,22.22,12.69,0.00,81.42,439.54,43.95,2021/10,360,False
4,3281.00,5.00,3.00,276.00,2021-03-16 18:05:25.233229,False,226.00,4.00,1690.00,PM5-2-4,SS & IRP,704.00,2021-02-02 11:11:57.575351,2021-03-17 12:35:12.690296,False,226.00,False,27.00,2020-11-23 14:05:30.101469,2021-09-29 14:53:35.625344,False,True,3.00,4.00,2.00,3.00,4.00,Smash your goals,The 10-session program created to help you bui...,360.00,2021-02-02 11:10:55.159616,2021-12-22 

In [101]:
df_session_exe.shape

imported_sessions.shape

imported_sessions = imported_sessions.sort_values(['id_users','updated_at_session_execution'])

imported_sessions_2 = imported_sessions[(imported_sessions['updated_at_session_execution'] > '2021-11-01')]


In [83]:
imported_sessions_lat_sess = imported_sessions.drop_duplicates(subset=['id_users'], keep = 'last')

# imported_sessions_lat_sess[(imported_sessions_lat_sess['imported'] == True) & (imported_sessions_lat_sess['updated_at_session_execution'].dt.year == 2022)]

# imported_sessions_lat_sess[(imported_sessions_lat_sess['imported'] == True) & (imported_sessions_lat_sess['updated_at_session_execution'] > '2021-11-01')]

# imported_sessions_lat_sess[(imported_sessions_lat_sess['imported'] == True) & (imported_sessions_lat_sess['updated_at_session_execution'] > '2021-11-01') & (imported_sessions_lat_sess['updated_at_session_execution'].dt.year == 2021)]

imported_sessions_lat_sess_users = imported_sessions_lat_sess[(imported_sessions_lat_sess['updated_at_session_execution'] > '2021-11-01')]

In [102]:
imported_sessions_2.sort_values('updated_at_session_execution')

,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,BMI,BMI_category,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories_session_execution,total_calories_users2,kcal_per_session_users2,YYYY/WW,id,imported
8389,6017.00,5.00,3.00,222.00,2021-11-01 00:06:11.953512,False,595.00,2.00,1238.00,W01_D02,Fundamentals - Session 2,21743.00,2021-10-28 06:17:47.167701,2021-11-19 22:35:44.327916,False,602.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,6557.00,2021-10-28 06:17:47.108449,2022-07-20 06:23:42.531597,male,1990-01-01,180.00,70.00,active,gain,thin,15.00,True,True,4.00,es,26447.00,True,5.00,21.60,Normal,40.00,1721.00,224.00,74140.00,7894.00,197.35,28.68,7.81,1.52,81.70,3878.08,96.95,2021/44,6557,True
8177,6018.00,4.00,4.00,120.00,2021-11-01 00:06:14.236997,False,865.00,3.00,630.00,Descanso_activo_3,Rewarding,26176.00,2021-10-28 15:35:59.277604,2021-10-28 15:35:59.277604,False,594.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,7761.00,2021-10-28 06:35:56.542124,2022-09-07 18:28:49.283551,male,1974-08-29,186.00,79.00,active,gain,mid,10.00,True,True,5.00,es,212090.00,False,37.00,22.84,Normal,135.00,1647.00,121.00,244626.00,36903.00,273.36,27.45,4.41,0.50,73.80,16250.29,120.37,2021/44,7761,True
8875,6019.00,5.00,3.00,302.00,2021-11-01 00:14:33.713258,False,537.00,2.00,1223.00,Sesion 2,Session 2,19842.00,2021-10-28 06:15:19.240653,2022-01-25 00:40:01.359275,True,565.00,False,29.00,2020-11-23 14:12:13.700871,2021-09-29 14:54:02.270887,True,True,3.00,4.00,3.00,4.00,4.00,Unbreakable,Join Unbreakable to optimize your body from th...,6000.00,2021-10-28 06:15:19.208652,2022-05-08 20:29:51.356913,male,1985-01-01,186.00,86.00,active,gain,mid,30.00,False,False,3.00,es,100309.00,False,8.00,24.86,Normal,28.00,1424.00,304.00,41367.00,6535.00,233.39,23.73,12.81,0.00,89.66,2900.87,103.60,2021/44,6000,True
6233,6020.00,5.00,3.00,313.00,2021-11-01 00:37:06.171780,False,322.00,9.00,1480.00,MuHu10.1,Session 8,22865.00,2021-10-28 06:19:14.772421,2021-11-19 02:34:49.321532,False,324.00,False,34.00,2020-11-23 14:15:43.775009,2021-09-29 14:54:09.125824,True,True,5.00,4.00,3.00,3.00,5.00,Muscle Hunters Initiation 1,Muscle Hunters is a program created for you to...,6899.00,2021-10-28 06:19:14.747586,2022-06-12 02:04:34.935740,male,1993-05-23,178.00,70.00,active,lose,mid,30.00,False,True,5.00,es,71762.00,False,5.00,22.09,Normal,27.00,1286.00,336.00,47207.00,5413.00,200.48,21.43,15.68,0.15,54.77,2659.88,98.51,2021/44,6899,True
18495,6021.00,6.00,4.00,140.00,2021-11-01 03:08:56.777633,False,764.00,1.00,494.00,PM1.P2.1_V2,Session 1,5816.00,2021-10-25 11:02:42.363208,2021-11-21 08:17:14.151481,False,773.00,True,23.00,2020-11-23 14:03:38.477856,2021-09-29 14:53:34.661221,False,True,1.00,3.00,1.00,2.00,2.00,Smash your goals,Choose 

In [103]:
imported_sessions_2.drop(['id', 'imported'], axis = 1, inplace = True)

In [104]:
imported_sessions_2 = imported_sessions_2.loc[:, 'id_session_execution']

In [105]:
df_after_upgrade = df.merge(imported_sessions_2, how = 'inner', left_on = 'id_session_execution', right_on = 'id_session_execution')


In [106]:
df_after_upgrade.shape

(1798735, 82)

In [66]:
'''

imported_sessions_lat_sess_users = imported_sessions_lat_sess_users.loc[:, ['id_session_execution']]

imported_sessions_lat_sess[(imported_sessions_lat_sess['updated_at_session_execution'] >= '2021-11-01')]['id_users']

df_after_upgrade = df.merge(imported_sessions_lat_sess_users, how = 'inner', left_on = 'id_session_execution', right_on = 'id_session_execution')

df_after_upgrade.shape # number of exercise executions after 1st deletion (1.11.2021)

df_session_after_upgrade = df_session_exe.merge(imported_sessions_lat_sess_users, how = 'inner', left_on = 'id_session_execution', right_on = 'id_session_execution')

df_session_after_upgrade.shape # number of sessions after 1st deletion (1.11.2021)

df_session_exe.shape

imported_sessions_lat_sess_users_other_date = imported_sessions_lat_sess[(imported_sessions_lat_sess['updated_at_session_execution'] >= '2021-12-01')]

df_after_upgrade_other_date = df.merge(imported_sessions_lat_sess_users_other_date, how = 'inner', left_on = 'id_users', right_on = 'id_users')

df_after_upgrade_other_date.shape # number of exercise executions after 2nd deletion (1.12.2021)

df_session_after_upgrade_other_date = df_session_exe.merge(imported_sessions_lat_sess_users_other_date, how = 'inner', left_on = 'id_users', right_on = 'id_users')

df_session_after_upgrade_other_date.shape # number of sessions after 2nd deletion (1.12.2021)


df_session_after_upgrade_users = df_session_after_upgrade.drop_duplicates(['id_users'], keep = 'last')

df_session_after_upgrade_users.shape # number of users after 1st deletion of data (1.11.2021)

df_after_upgrade_other_date_users = df_after_upgrade_other_date.drop_duplicates(['id_users'], keep = 'last')

df_after_upgrade_other_date_users.shape # number of users after 2nd deletion (1.12.2021)

# looking only for data without imported users
imported_only_false = imported_all[imported_all['imported'] == False]

imported_only_false = imported_only_false.sort_values(['id_users','updated_at_session_execution'])

imported_only_false_sess_exe = imported_only_false.drop_duplicates(['id_session_execution'], keep = 'last')

imported_only_false_sess_exe.shape # number of sessions after deleting all users that were in a new app

imported_only_false_users = imported_only_false.drop_duplicates(['id_users'], keep = 'last')

#imported_only_false_users.shape # number of users after deleting all users that were in a new app

'''

"\n\nimported_sessions_lat_sess_users = imported_sessions_lat_sess_users.loc[:, ['id_session_execution']]\n\nimported_sessions_lat_sess[(imported_sessions_lat_sess['updated_at_session_execution'] >= '2021-11-01')]['id_users']\n\ndf_after_upgrade = df.merge(imported_sessions_lat_sess_users, how = 'inner', left_on = 'id_session_execution', right_on = 'id_session_execution')\n\ndf_after_upgrade.shape # number of exercise executions after 1st deletion (1.11.2021)\n\ndf_session_after_upgrade = df_session_exe.merge(imported_sessions_lat_sess_users, how = 'inner', left_on = 'id_session_execution', right_on = 'id_session_execution')\n\ndf_session_after_upgrade.shape # number of sessions after 1st deletion (1.11.2021)\n\ndf_session_exe.shape\n\nimported_sessions_lat_sess_users_other_date = imported_sessions_lat_sess[(imported_sessions_lat_sess['updated_at_session_execution'] >= '2021-12-01')]\n\ndf_after_upgrade_other_date = df.merge(imported_sessions_lat_sess_users_other_date, how = 'inner', l

In [108]:
df_after_upgrade['id_session_execution'].nunique()

47617

In [109]:
df26 = df_after_upgrade.copy()

In [110]:
df26.drop(['total_sessions_users2',
       'total_time_session_execution', 'total_reps_session_execution',
       'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
       'total_time_session_execution_min', 'reps_per_min_session_execution',
       'exercise_execution_time_min', 'calories', 'calories_session_execution',
       'total_calories_users2', 'kcal_per_session_users2'], axis = 1, inplace = True)

In [111]:
cos6 = df26.loc[:, ['id_users', 'id_session_execution']]

In [112]:
total_sessions_user2 = cos6.groupby(['id_users'])['id_session_execution'].nunique()

In [113]:
total_sessions_user2

id_users
108.00       7
112.00       1
172.00       6
219.00       2
514.00      18
            ..
18127.00     1
18147.00     1
18157.00     1
18165.00     3
18174.00     1
Name: id_session_execution, Length: 3034, dtype: int64

In [114]:
df27 = df26.copy()

In [115]:
df27 = df27.merge(total_sessions_user2, how = 'left', left_on = 'id_users', right_on = 'id_users')

In [116]:
df27.rename(columns = {'id_session_execution_x': 'id_session_execution',
                       'id_session_execution_y': 'total_sessions_users2'},
            inplace = True)

In [117]:
df27['total_sessions_users2'] = df27['total_sessions_users2'].astype(float)

In [118]:
cos7 = df27.groupby(['id_session_execution'])['reps_executed_ex_ex'].sum()

cos8 = df27.groupby(['id_session_execution'])['execution_time_ex_ex'].sum()

In [119]:
df28 = df27.merge(cos8, how = 'left', left_on = 'id_session_execution', right_on = 'id_session_execution')

df28.rename(columns = {'execution_time_ex_ex_x': 'execution_time_ex_ex',
                       'execution_time_ex_ex_y': 'total_time_session_execution'},
           inplace = True)

In [120]:
df29 = df28.merge(cos7, how = 'left', left_on = 'id_session_execution', right_on = 'id_session_execution')

df29.rename(columns = {'reps_executed_ex_ex_x': 'reps_executed_ex_ex',
                       'reps_executed_ex_ex_y': 'total_reps_session_execution'},
           inplace = True)

In [121]:
coss1 = df29.drop_duplicates(subset=['id_session_execution'], keep='first')

cos9 = coss1.groupby(['id_users'])['total_time_session_execution'].sum()

In [122]:
df30 = df29.merge(cos9, how = 'left', left_on = 'id_users', right_on = 'id_users')

df30.rename(columns = {'total_time_session_execution_x': 'total_time_session_execution',
                       'total_time_session_execution_y': 'total_time_users2'},
            inplace = True)

In [123]:
cos10 = coss1.groupby(['id_users'])['total_reps_session_execution'].sum()

In [124]:
df31 = df30.merge(cos10, how = 'left', left_on = 'id_users', right_on = 'id_users')

df31.rename(columns = {'total_reps_session_execution_x': 'total_reps_session_execution',
                       'total_reps_session_execution_y': 'total_reps_users2'},
            inplace = True)

In [125]:
df31['reps_per_session_users2'] = df31['total_reps_users2'] / df31['total_sessions_users2']

In [126]:
df31['total_time_session_execution_min'] = df31['total_time_session_execution']/60

df31['reps_per_min_session_execution'] = df31['total_reps_session_execution']/df31['total_time_session_execution_min']


In [127]:
df31['exercise_execution_time_min'] = df31['execution_time_ex_ex']/60

df31['calories'] = df31['exercise_execution_time_min'] * (df31['met_multiplier_exercises'] * 3.5 * df31['weight']) / 200


In [128]:
cos11 = df31.groupby(['id_session_execution'])['calories'].sum()

In [129]:
df32 = df31.merge(cos11, how = 'left', left_on = 'id_session_execution', right_on = 'id_session_execution')

df32.rename(columns = {'calories_x': 'calories',
                       'calories_y': 'calories_session_execution'},
            inplace = True)

In [130]:
coss2 = df32.drop_duplicates(subset=['id_session_execution'], keep='first')

cos12 = coss2.groupby(['id_users'])['calories_session_execution'].sum()

In [131]:
df33 = df32.merge(cos12, how = 'left', left_on = 'id_users', right_on = 'id_users')

df33.rename(columns = {'calories_session_execution_x': 'calories_session_execution',
                       'calories_session_execution_y': 'total_calories_users2'},
            inplace = True)

In [132]:
df33['kcal_per_session_users2'] = df33['total_calories_users2'] / df33['total_sessions_users2']

In [133]:
df33.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2
0,1296434,0.00,2.00,3,2021-11-01 12:05:39.918185,53220.00,3.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.03,0.04,50.95,2463.91,48.31
1,1296435,3.00,8.00,1,2021-11-01 12:05:39.928907,53221.00,4.00,5398,2020-10-15 12:37:17.966340,2021-10-13 09:46:29.957461,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (right),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.13,0.54,50.95,2463.91,48.31
2,1296436,3.00,9.00,2,2021-11-01 12:05:39.933210,53221.00,4.00,5399,2020-10-15 12:37:17.976799,2021-10-13 09:46:29.967689,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (left),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,36

In [134]:
df = df33.copy()

In [135]:
cols = ['name_en_exercises','discarded_session_execution', 'code_name_sessions', 
        'name_en_sessions', 'active_user_programs', 'completed_user_programs',
        'pro_programs', 'available_programs', 'name_en_programs', 
        'gender', 
        'activity_level', 'goal', 'body_type', 'newsletter_subscription', 'notifications_setting',
        'language', 'scientific_data_usage', 'BMI_category', 'name_en_implements']

for col in cols:
    df[col] = df[col].astype('category')
    
col_date = ['updated_at_ex_ex', 'created_at_exercises', 'updated_at_exercises',
            'updated_at_session_execution', 'created_at_user_programs', 'updated_at_user_programs',
            'created_at_programs', 'updated_at_programs', 'created_at_users', 'updated_at_users',
            'date_of_birth']

for col in col_date:
    df[col] = pd.to_datetime(df[col])#.dt.strftime("%Y-%m-%d %H:%M:%S")

cat_names = {
    'gender': {1: 'male', 0: 'female'},
    'activity_level': {0: 'very active', 1: 'active', 2: 'sedentary'},
    'goal': {0: 'lose', 1: 'gain', 2: 'antiaging'},
    'body_type': {0: 'thin', 1: 'mid', 2: 'strong'}
}

df = df.replace(cat_names)


In [136]:
df['total_sessions_users2'].max()

303.0

In [137]:
exercise_per_sess = pd.DataFrame(df.groupby(['id_session_execution'])['id_session_execution'].count())

exercise_per_sess.rename(columns = {'id_session_execution': 'number_exercises_in_session'}, inplace = True)

exercise_per_sess.reset_index(inplace = True)

df_no_rest = df[df['name_en_exercises'] != 'Rest']

exercise_per_sess_no_rest = pd.DataFrame(df_no_rest.groupby(['id_session_execution'])['id_session_execution'].count())

exercise_per_sess_no_rest.rename(columns = {'id_session_execution': 'number_exercises_without_rest_in_session'}, inplace = True)

exercise_per_sess_no_rest.reset_index(inplace = True)

df = df.merge(exercise_per_sess, how = 'left', left_on = 'id_session_execution', right_on = 'id_session_execution')

df = df.merge(exercise_per_sess_no_rest, how = 'left', left_on = 'id_session_execution', right_on = 'id_session_execution')

In [138]:
'''
df.drop(['number_exercises_without_rest_in_session_y', 'number_exercises_in_session_y'], inplace = True, axis = 1)

df.rename(columns = {'number_exercises_without_rest_in_session_x': 'number_exercises_without_rest_in_session',
                     'number_exercises_in_session_x': 'number_exercises_in_session'},
         inplace = True)

'''


exercises_per_user = pd.DataFrame(df.groupby(['id_users'])['number_exercises_in_session'].sum())

exercises_per_user.rename(columns = {'number_exercises_in_session': 'exercises_per_user'}, inplace = True)

exercises_per_user.reset_index(inplace = True)

df = df.merge(exercises_per_user, how = 'left', left_on = 'id_users', right_on = 'id_users')

In [139]:
'''
df.drop(['exercises_per_user_y'], axis = 1, inplace = True)
df.rename(columns = {'exercises_per_user_x': 'exercises_per_users'},
         inplace = True)
'''

exercises_no_rest_per_user = pd.DataFrame(df.groupby(['id_users'])['number_exercises_without_rest_in_session'].sum())

exercises_no_rest_per_user.rename(columns = {'number_exercises_without_rest_in_session': 'exercises_without_rest_per_user'}, inplace = True)

exercises_no_rest_per_user.reset_index(inplace = True)

df = df.merge(exercises_no_rest_per_user, how = 'left', left_on = 'id_users', right_on = 'id_users')

In [140]:
df.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,number_exercises_in_session,number_exercises_without_rest_in_session,exercises_per_user,exercises_without_rest_per_user
0,1296434,0.00,2.00,3,2021-11-01 12:05:39.918185,53220.00,3.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.03,0.04,50.95,2463.91,48.31,55,32,52898,40465
1,1296435,3.00,8.00,1,2021-11-01 12:05:39.928907,53221.00,4.00,5398,2020-10-15 12:37:17.966340,2021-10-13 09:46:29.957461,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (right),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.13,0.54,50.95,2463.91,48.31,55,32,52898,40465
2,1296436,3.00,9.00,2,2021-11-01 12:05:39.933210,53221.00,4.00,5399,2020-10-15 12:37:17.976799,2021-10-13 09:46:29.967689,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (left),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:

In [141]:
df_users_only_once = df.drop_duplicates(subset=['id_users'], keep='first')

In [142]:
df_users_only_once.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,number_exercises_in_session,number_exercises_without_rest_in_session,exercises_per_user,exercises_without_rest_per_user
0,1296434,0.00,2.00,3,2021-11-01 12:05:39.918185,53220.00,3.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.03,0.04,50.95,2463.91,48.31,55,32,52898,40465
55,1301839,3.00,9.00,1,2021-11-02 06:34:48.954209,54970.00,7.00,5398,2020-10-15 12:37:17.966340,2021-10-13 09:46:29.957461,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (right),8692.00,3.00,6293.00,7.00,4.00,368.00,2021-11-02 06:34:49.022130,False,685.00,2.00,1105.00,PH1.2_V2,Session 2,26994.00,2021-11-01 08:52:54.714380,2021-11-07 07:34:19.999433,False,688.00,True,6.00,2020-11-23 11:28:22.066874,2021-09-29 14:53:41.901478,False,True,1.00,2.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program that helps you...,3466.00,2021-10-28 06:01:36.983684,2021-12-22 06:41:24.982549,male,1980-01-16,179.00,86.00,sedentary,lose,thin,45.00,True,True,4.00,es,140321.00,False,1.00,0.00,No implement,26.84,Overweight,2021/44,5.00,937.00,368.00,2847.00,850.00,170.00,15.62,23.56,0.15,0.79,54.20,188.45,37.69,55,32,5028,3618
110,1302658,10.00,65.00,1,2021-11-02 07:30:39.314452,55265.00,1.00,5279,2020-10-15 12:37:16.490561,2021-10-13 09:46:29.275591,"{""Todo el cuerpo""}","{pectorales,"" tríceps"","" cuádriceps""}","{hombros,"" cadera""}",3.80,Semi burpee,8754.00,1.00,6313.00,6.00,4.00,140.00,2021-11-02 07:30:39.446576,False,764.00,1.00,494.00,PM1.P2.1_V2,Session 1,26835.00,2021-10-31 14:36:50.033053,2022-03-28 09:36:57.673985,False,773.00,True,23.00,2020-11-23 14:03:38.477856,2021-09-29 14:53:34.661221,False,True,1.00,3.00,1.00,2.00,2.00,Smash your goals,Choose the program created to help you get int...,1288.00,2021-10-25 11:00:14.208971,2022-

In [143]:
df.to_csv(r'/home/evida-monika/mhunters/final_data_4.csv'+ '.bz2', header=True, index=False,compression='bz2')
